# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
pd.options.display.max_rows = 40
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df_group = data.groupby(["CustomerID", "ProductName"]).agg({'Quantity': 'sum'})
df_group

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
product_matrix = pd.pivot_table(data, values="Quantity", index=["ProductName"], columns=['CustomerID'], aggfunc='sum', fill_value=0)
print(product_matrix.shape)
product_matrix

(452, 1000)


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
eu_dist = pdist(product_matrix.T, 'euclidean')
eu_dist = squareform(eu_dist)
# Convertir a dataframe y añadirle el id del cliente a filas y columnas
eu_distance = pd.DataFrame(eu_dist, index=product_matrix.columns, columns=product_matrix.columns)
eu_distance

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [34]:
# Clientes similares a Susan Stevenson, con ID: 40094
similarities_Susan = eu_distance[40094].sort_values(ascending=False)[:5]
similarities_Susan

CustomerID
92255    264.505198
95017    255.693176
92492    252.719212
92525    252.529206
87920    252.081336
Name: 40094, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [35]:
# Extraemos los índices de los clientes similares de Susan y hacemos el subset con ellos
similares = df_group.loc[similarities_Susan.index,:]
similares

Quantity
CustomerID ProductName                           
87920      Apricots - Dried                    23
           Apricots Fresh                      23
           Bagel - Plain                       23
           Beef - Short Loin                   23
           Beef Ground Medium                  23
...                                           ...
95017      Vol Au Vents                        25
           Whmis - Spray Bottle Trigger        50
           Wine - Redchard Merritt             25
           Wine - Toasted Head                 25
           Wine - White Cab Sauv.on            25

[354 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [36]:
# Como ya tenía los datos sumados en la columna Quantity en la tabla 1, es lo mismo que se consigue en el paso 5.
ranked = similares.sort_values(by='Quantity', ascending= False)
ranked

Quantity
CustomerID ProductName                               
95017      Coffee - Irish Cream                    75
92525      Wine - Chablis 2003 Champs              72
87920      Pork - Hock And Feet Attached           69
           Beer - Original Organic Lager           69
           Soup Campbells - Italian Wedding        69
...                                               ...
           Peas - Pigeon, Dry                      23
           Pastry - Choclate Baked                 23
           Oregano - Dry, Rubbed                   23
           Muffin Batt - Blueberry Passion         23
           Apricots - Dried                        23

[354 rows x 1 columns]

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [37]:
# Unir los dos data frames
merged = ranked.merge(product_matrix, how= "left", left_on='ProductName', right_on='ProductName')
merged

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Coffee - Irish Cream,75,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Wine - Chablis 2003 Champs,72,0,0,1,0,0,0,0,0,0,...,25,0,0,0,0,25,0,0,0,0
Pork - Hock And Feet Attached,69,1,0,1,0,0,0,0,1,0,...,0,25,0,0,0,25,0,0,0,0
Beer - Original Organic Lager,69,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Soup Campbells - Italian Wedding,69,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Peas - Pigeon, Dry",23,0,1,0,0,0,0,0,1,0,...,0,25,25,0,0,25,0,0,0,0
Pastry - Choclate Baked,23,0,0,0,0,0,0,1,1,1,...,0,0,25,0,0,0,0,25,0,0
"Oregano - Dry, Rubbed",23,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# Filtrar por productos que Susan Stevenson no haya comprado
nocomprados = merged[merged[40094]==0]
# Los 5 más comprados pero que Susan nunca los ha pillado
nocomprados[0:5]

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Coffee - Irish Cream,75,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Wine - Chablis 2003 Champs,72,0,0,1,0,0,0,0,0,0,...,25,0,0,0,0,25,0,0,0,0
Pork - Hock And Feet Attached,69,1,0,1,0,0,0,0,1,0,...,0,25,0,0,0,25,0,0,0,0
Beer - Original Organic Lager,69,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Soup Campbells - Italian Wedding,69,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,25


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [42]:
# Extraer la lista de clientes
lista_customers = [e for e in data.CustomerID]


In [46]:
# Creamos la función
def recomendador(cliente):
    five_similars = (eu_distance[cliente].sort_values(ascending=False)[:5])    
    ranked = (df_group.loc[five_similars.index,:]).sort_values(by='Quantity', ascending= False)
    merged = ranked.merge(product_matrix, how= "left", left_on='ProductName', right_on='ProductName')
    # Filtrar por productos que Susan Stevenson no haya comprado
    nocomprados = merged[merged[cliente]==0]
    # Los 5 más comprados pero que el cliente nunca los ha pillado
    nocomprados[0:5][cliente]
    return dic_rec.update({cliente:{
                  "Recom_1": nocomprados.index[0],
                  "Recom_2": nocomprados.index[1],
                  "Recom_3": nocomprados.index[2],
                  "Recom_4": nocomprados.index[3],
                  "Recom_5": nocomprados.index[4]}})
    


In [47]:
# Pasamos la función a la lista de clientes y vamos almacenando los valores en el diccionario
dic_rec = {}
# La voy a ejecutar sólo para los 30 primeros clientes porque se me va la vida esperando
for e in lista_customers[:30]:
    recomendador(e)


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [48]:
data_rec = pd.DataFrame(dic_rec)
data_rec

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,3267,19260,76530,30196,48818,15166,11443,79060,82597,62411
Recom_1,Tomato - Tricolor Cherry,Coffee - Irish Cream,Coffee - Irish Cream,Coffee - Irish Cream,Tahini Paste,Coffee - Irish Cream,Coffee - Irish Cream,Coffee - Irish Cream,Coffee - Irish Cream,Coffee - Irish Cream,...,Coffee - Irish Cream,Coffee - Irish Cream,Wine - Chablis 2003 Champs,Coffee - Irish Cream,Coffee - Irish Cream,Coffee - Irish Cream,Coffee - Irish Cream,Spice - Peppercorn Melange,Longos - Grilled Salmon With Bbq,Coffee - Irish Cream
Recom_2,"Pasta - Penne, Rigate, Dry",Bandage - Flexible Neon,Wine - Chablis 2003 Champs,Wine - Chablis 2003 Champs,Pate - Cognac,Tahini Paste,Tomato - Tricolor Cherry,Pork - Hock And Feet Attached,Wine - Chablis 2003 Champs,Tomato - Tricolor Cherry,...,Tomato - Tricolor Cherry,Soup Campbells - Italian Wedding,"Beans - Kidney, Red Dry",Wine - Chablis 2003 Champs,Spice - Peppercorn Melange,Soup Campbells - Italian Wedding,Wine - Chablis 2003 Champs,Coffee - Irish Cream,Scampi Tail,Scallops - 10/20
Recom_3,"Salsify, Organic",Whmis - Spray Bottle Trigger,Pork - Hock And Feet Attached,"Turnip - White, Organic",Quiche Assorted,Pate - Cognac,Bandage - Flexible Neon,Beer - Original Organic Lager,"Turnip - White, Organic",Whmis - Spray Bottle Trigger,...,Whmis - Spray Bottle Trigger,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Bandage - Flexible Neon,Wine - Chablis 2003 Champs,Beer - Original Organic Lager,"Turnip - White, Organic",Soup Campbells - Italian Wedding,Scallops - 10/20,Muffin Chocolate Individual Wrap
Recom_4,Scallops - Live In Shell,Bagel - Plain,Beer - Original Organic Lager,Bandage - Flexible Neon,Bread Crumbs - Panko,Ecolab - Lime - A - Way 4/4 L,Onion Powder,Soup Campbells - Italian Wedding,Cream Of Tartar,Bandage - Flexible Neon,...,Bandage - Flexible Neon,Pork - Hock And Feet Attached,"Beef - Ground, Extra Lean, Fresh",Pastry - Butterscotch Baked,Beer - Original Organic Lager,Pork - Hock And Feet Attached,"Pasta - Penne, Rigate, Dry",Beer - Original Organic Lager,Soup - Campbells Bean Medley,Bandage - Flexible Neon
Recom_5,Whmis - Spray Bottle Trigger,Mussels - Cultivated,Soup Campbells - Italian Wedding,Muffin Chocolate Individual Wrap,Cheese - Parmesan Grated,Cheese - Parmesan Grated,"Turnip - White, Organic",Pastry - Butterscotch Baked,Bandage - Flexible Neon,"Turnip - White, Organic",...,Onion Powder,Onion Powder,Tea - Herbal Sweet Dreams,Onion Powder,Pork - Hock And Feet Attached,Onion Powder,Cream Of Tartar,Pork - Hock And Feet Attached,Pecan Raisin - Tarts,"Beef - Tenderlion, Center Cut"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [49]:
# Lo mismo, pero con la similaridad del coseno
cos_dist = squareform(pdist(product_matrix.T, 'cosine'))
# Convertir a dataframe y añadirle el id del cliente a filas y columnas
cos_distance = pd.DataFrame(cos_dist, index=product_matrix.columns, columns=product_matrix.columns)
cos_distance

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,0.886781,0.774648,0.839772,0.843681,0.848243,0.915650,0.884009,0.875835,0.868135,...,0.885703,0.930549,0.895154,0.955144,0.872026,0.946249,0.891093,0.922307,0.888643,0.897020
200,0.886781,0.000000,0.861622,0.868815,0.918555,0.926067,0.795510,0.935250,0.864456,0.846889,...,0.834109,0.875937,0.801003,0.839743,0.885697,0.915985,0.744661,0.861214,0.900539,0.896524
264,0.774648,0.861622,0.000000,0.879829,0.817627,0.903428,0.855401,0.942005,0.862039,0.907695,...,0.828554,0.902768,0.868942,0.880384,0.808039,0.784998,0.823027,0.857563,0.802031,0.897020
356,0.839772,0.868815,0.879829,0.000000,0.901204,0.908448,0.862918,0.848803,0.934606,0.924994,...,0.864555,0.920991,0.875755,0.872427,0.890811,0.885348,0.922566,0.864967,0.835782,0.865764
412,0.843681,0.918555,0.817627,0.901204,0.000000,0.872402,0.843972,0.825672,0.885161,0.841452,...,0.894287,0.768755,0.890907,0.889368,0.846130,0.863288,0.924455,0.856285,0.828344,0.809510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.946249,0.915985,0.784998,0.885348,0.863288,0.842051,0.850545,0.875503,0.868375,0.899353,...,0.877321,0.854225,0.912473,0.843081,0.755807,0.000000,0.779202,0.839396,0.881952,0.926312
98069,0.891093,0.744661,0.823027,0.922566,0.924455,0.919993,0.871885,0.901903,0.879989,0.847056,...,0.889526,0.865745,0.885994,0.927741,0.802092,0.779202,0.000000,0.824781,0.844531,0.863140
98159,0.922307,0.861214,0.857563,0.864967,0.856285,0.873163,0.867060,0.866702,0.835112,0.866645,...,0.842378,0.795676,0.867460,0.917521,0.847048,0.839396,0.824781,0.000000,0.840744,0.881655


In [51]:
# Cambiamos cuatro cosillas de la función
def recomendador_cos(cliente):
    five_similars = (cos_distance[cliente].sort_values(ascending=False)[:5])    
    ranked = (df_group.loc[five_similars.index,:]).sort_values(by='Quantity', ascending= False)
    merged = ranked.merge(product_matrix, how= 'left',left_on='ProductName', right_on='ProductName')
    # Filtrar por productos que Susan Stevenson no haya comprado
    nocomprados = merged[merged[cliente]==0]
    # Los 5 más comprados pero que el cliente nunca los ha pillado
    nocomprados[0:5][cliente]
    return dic_rec_cos.update({cliente:{
                  "Recom_1": nocomprados.index[0],
                  "Recom_2": nocomprados.index[1],
                  "Recom_3": nocomprados.index[2],
                  "Recom_4": nocomprados.index[3],
                  "Recom_5": nocomprados.index[4]}})

In [52]:
# Pasamos la función a la lista de clientes y vamos almacenando los valores en el diccionario
dic_rec_cos = {}
# La voy a ejecutar sólo para los 30 primeros clientes porque se me va la vida esperando
for e in lista_customers[:30]:
    recomendador_cos(e)

In [53]:
data_rec_cos = pd.DataFrame(dic_rec_cos)
data_rec_cos

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,3267,19260,76530,30196,48818,15166,11443,79060,82597,62411
Recom_1,Milk - 2%,Wine - Fume Blanc Fetzer,Bread - English Muffin,Chocolate - Feathers,Lamb - Ground,Salmon - Sockeye Raw,Ecolab - Lime - A - Way 4/4 L,"Appetizer - Mini Egg Roll, Shrimp","Soup - Campbells, Cream Of",Blackberries,...,Blackberries,Cheese - Camembert,Chicken - Soup Base,Tray - 16in Rnd Blk,Cheese - Cottage Cheese,Smoked Paprika,Flavouring - Orange,Soup - Campbells Bean Medley,Water - Aquafina Vitamin,Wine - Vidal Icewine Magnotta
Recom_2,Onions - Cippolini,Carbonated Water - Blackcherry,Sugar - Fine,Yeast Dry - Fermipan,Wine - Pinot Noir Latour,Wonton Wrappers,"Liners - Banana, Paper",Napkin White - Starched,Wine - Crozes Hermitage E.,Ocean Spray - Kiwi Strawberry,...,Lettuce - California Mix,Rambutan,Tea - Herbal Sweet Dreams,"Wine - Red, Cooking",Wine - Crozes Hermitage E.,Pie Filling - Cherry,Snapple - Iced Tea Peach,Milk - 1%,Ice Cream Bar - Oreo Cone,"Wine - Cahors Ac 2000, Clos"
Recom_3,"Chicken - Leg, Boneless",Soupcontfoam16oz 116con,Foam Cup 6 Oz,Ecolab - Mikroklene 4/4 L,French Pastry - Mini Chocolate,Cake - Cake Sheet Macaroon,Gatorade - Xfactor Berry,Beef - Texas Style Burger,Cheese - Cottage Cheese,Gatorade - Xfactor Berry,...,"Cheese - Brie,danish","Water, Tap",Cream Of Tartar,Tea - Jasmin Green,Wine - Vidal Icewine Magnotta,Wonton Wrappers,Wine - Hardys Bankside Shiraz,Soup Knorr Chili With Beans,Coffee - Dark Roast,"Pork - Loin, Bone - In"
Recom_4,Lime Cordial - Roses,Tea - English Breakfast,"Lamb - Pieces, Diced",Soup - Campbells Tomato Ravioli,"Turnip - White, Organic","Turnip - White, Organic","Salsify, Organic","Sauce - Gravy, Au Jus, Mix",Wine - Alsace Gewurztraminer,"Beans - Kidney, Canned",...,Beer - Rickards Red,Vanilla Beans,Ecolab - Lime - A - Way 4/4 L,Wanton Wrap,Brandy - Bar,Beef - Montreal Smoked Brisket,Soup Knorr Chili With Beans,"Sauce - Gravy, Au Jus, Mix",Apricots - Halves,Cookie Dough - Double
Recom_5,Lettuce - Treviso,Pasta - Orecchiette,Bread - Italian Corn Meal Poly,Lime Cordial - Roses,Bread - Italian Corn Meal Poly,Wine - Ruffino Chianti,Pastry - Choclate Baked,Sage - Ground,Nantuket Peach Orange,Lime Cordial - Roses,...,Beans - Wax,"Turkey - Whole, Fresh",Bread - Bistro White,Bandage - Flexible Neon,Bread - Rye,"Chestnuts - Whole,canned",Cheese - Mozzarella,"Pepsi - Diet, 355 Ml",Gatorade - Xfactor Berry,Crush - Cream Soda
